# Разметка семантических ролей (SRL)

## Improrts

In [ ]:
!pip3 install grpcio
!pip3 install git+https://github.com/RuslanKorchenov/isanlp.git
!git clone https://github.com/IINemo/isanlp_srl_framebank.git
!pip3 install gensim==3.6.0 smart-open==1.7.0 numpy tensorflow==1.15.0 sklearn

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git-lfs pull

In [ ]:
!pip3 install bert-serving-client
!pip3 install deeppavlov
!pip3 install tensorflow-hub

In [1]:
!pip3 install tensorflow-gpu==1.15

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [1]:
from isanlp_srl_framebank.processor_srl_framebank import ProcessorSrlFramebank

[nltk_data] Downloading package punkt to /home/rusel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rusel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/rusel/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/rusel/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [2]:
srl_proc = ProcessorSrlFramebank(model_dir_path = 'isanlp_srl_framebank/models', known_preds_embeddings_type = "elmo")




INFO:tensorflow:Saver not created because there are no variables in the graph to restore



In [1]:
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd
from isanlp import PipelineCommon

from isanlp.processor_syntaxnet_remote import ProcessorSyntaxNetRemote



# - To start morphology processor:
# docker run --rm -d -p  3333:3333 inemo/isanlp
# - To start ProcessorSyntaxNetRemote:
# docker run --rm -d --shm-size=1024m -ti -p 8111:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
# - To start SRL processor:
# docker run --rm -d -p  3334:3333 inemo/isanlp_srl_framebank
syntax_ppl = PipelineCommon([(ProcessorRemote('localhost', 3333, 'default'), 
                              ['text'], 
                              {'sentences' : 'sentences', 
                               'tokens' : 'tokens',
                               'postag' : 'mystem_postags',
                               'lemma' : 'lemma'}),
                              (ProcessorSyntaxNetRemote('localhost', 8111), 
                               ['tokens', 'sentences'], 
                               {'syntax_dep_tree' : 'syntax_dep_tree', 
                                'morph' : 'morph_syntaxnet'}),
                              (ConverterMystemToUd(), 
                               ['mystem_postags'], 
                               {'morph' : 'morph',
                                'postag' : 'postag'})])

In [11]:
syntax_ppl = PipelineCommon([(ProcessorRemote('localhost', 3333, 'default'), 
                              ['text'], 
                              {'lemma' : 'lemma'}),])

In [12]:
text = "Как отводятся излишки тепла у млекопитающих?"

In [13]:
annots = syntax_ppl(text)

In [14]:
annots

{'text': 'Как отводятся излишки тепла у млекопитающих?',
 'lemma': [['как',
   'отводиться',
   'излишек',
   'тепло',
   'у',
   'млекопитающее',
   '?']]}

In [7]:
annots_srl = srl_proc(annots['tokens'],
                  annots['postag'],   
                  annots['morph'], 
                  annots['lemma'], 
                  annots['syntax_dep_tree'])

In [8]:
annots['srl'] = annots_srl

In [9]:
def print_roles(lemma, role_annot):
    for sent_num, ann_sent in enumerate(role_annot):
        for event in ann_sent:
            print('=====Pred: {}'.format(lemma[sent_num][event.pred[0]]))
            for arg in event.args:
                print('Arg({}): {}'.format(arg.tag, lemma[sent_num][arg.begin]))

In [10]:
print_roles(annots['lemma'], annots['srl'])

=====Pred: отводиться
Arg(пациенс): излишек
Arg(место): млекопитающее


In [2]:
import numpy as np

train = np.load('train.npy',allow_pickle=True)
dev = np.load('dev.npy',allow_pickle=True)

In [7]:
train[1]

{'context': 'По данным рейтинга Top 100 самых филиальных банков России агентства РБК.Рейтинг УБРиР на 29 ноября 2013 года занял 9 место. По данным рейтинга РБК.Рейтинг УБРиР вошел в 20 крупнейших банков России по объёму потребительского кредитования. По данным рейтинга Интерфакс 100 УБРиР вошел в число крупнейших банков России по основным показателям деятельности по итогам деятельности в 2013г — 34 место по размеру активов, 22 место по объёму средств частных лиц. ПАО КБ УБРиР регулярно входит в список крупнейших участников фондового рынка, составляемых крупнейшей торговой площадкой страны — Московской биржей. По данным портала banki.ru на 1 февраля 2017 года УБРиР занимает: 19 место по вкладам физических лиц; 42 место по кредитному портфелю физических лиц; 40 место по собственному капиталу. Журнал Forbes по итогам 1 квартала 2014 года: 48 место в рейтинге 100 самых надёжных банков России .',
 'question': 'Какое место занимает УБРиР по даным рейтинга Top 100 самых филиальных банков Росс

In [10]:
print("Length Train:", len(train))
print("Length Train:", len(dev))

Length Train: 45328
Length Train: 5036


## Разметка

Парсим все данные и сохраняем файлами по 100

In [ ]:
from tqdm import tqdm
import numpy as np

def ann(text):
    annots = syntax_ppl(text)
    annots_srl = srl_proc(annots['tokens'],
                  annots['postag'],   
                  annots['morph'], 
                  annots['lemma'], 
                  annots['syntax_dep_tree'])
    annots['srl'] = annots_srl
    return annots

for j in range(0, 453):
    srl_train=[]
    s = 0 + j*100
    train_temp = train[s:s+100]
    for i in tqdm(train_temp):
        srl_temp={'context':ann(i['context']), 'question':ann(i['question'])}
        srl_train.append(srl_temp)
    np.save('final/train1_'+str(s), srl_train)

Соединяем все файлы

In [ ]:
full = []

for i in tqdm(range(0, 453)):
    temp_load = np.load('final/train1_' + str(i) + '00.npy', allow_pickle=True)
    full_train = np.append(full_train, temp_load)

In [ ]:
np.save('final/train_framebank', full_train)

Проводим ту же процедуру для набора данных dev

In [ ]:
for j in range(0, 50):
    srl_dev=[]
    s = 0 + j*100
    dev_temp = dev[s:s+100]
    for i in tqdm(dev_temp):
        srl_temp={'context':ann(i['context']), 'question':ann(i['question'])}
        srl_train.append(srl_temp)
    np.save('final/dev1_'+str(s), srl_dev)

In [ ]:
full = []

for i in tqdm(range(0, 50)):
    temp_load = np.load('final/dev1_' + str(i) + '00.npy', allow_pickle=True)
    full_dev = np.append(full_dev, temp_load)

In [ ]:
np.save('final/dev_framebank', full_dev)